In [1]:
%cd ../..

/home/matheus/Desktop/Itens/Projetos/llm2vec-embeddings-classification


In [21]:
import os
import numpy as np
import pandas as pd
from src.core.utils import read_json

# Configura o Pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

def load_results_to_dataframe(base_path: str) -> pd.DataFrame:
    """
    Load results from JSON files into a pandas DataFrame.
    """
    results = []
    
    # Traverse the directory structure
    for dataset_name in os.listdir(base_path):
        dataset_path = os.path.join(base_path, dataset_name)
        if os.path.isdir(dataset_path):
            for model_type in os.listdir(dataset_path):
                model_type_path = os.path.join(dataset_path, model_type)
                
                if os.path.isdir(model_type_path):
                    for model_name in os.listdir(model_type_path):
                        if model_name == "phi3.5:3.8b":
                            continue
                        model_name_path = os.path.join(model_type_path, model_name)
                        
                        # Define paths based on whether prompt_name is needed
                        if model_type != "bert":
                            subdirs = [os.path.join(model_name_path, prompt) for prompt in os.listdir(model_name_path)]
                        else:
                            subdirs = [model_name_path]
                        
                        # Process results.json files from determined paths
                        for subdir in subdirs:
                            for classifier in os.listdir(subdir):

                                classifier_path = os.path.join(subdir, classifier)
                                
                                # Check for the results.json in the classifier path
                                json_file_path = os.path.join(classifier_path, 'results.json')
                                
                                if os.path.isfile(json_file_path):
                                    result_data = read_json(json_file_path)

                                    keys_to_extract = ["split0_test_f1_score", "split1_test_f1_score", "split2_test_f1_score", "split3_test_f1_score", "split4_test_f1_score", "mean_test_f1_score", "embedding_generation_time", "embedding_generation_size"]
    
                                    # Extrai apenas as chaves especificadas
                                    result_data= {key: result_data.get(key) for key in keys_to_extract}
                                    
                                    # Add metadata to the result data
                                    result_data['dataset_name'] = dataset_name
                                    result_data['model_type'] = model_type
                                    result_data['model_name'] = model_name
                                    result_data['classifier'] = classifier
                                    
                                    # Add prompt_name if applicable
                                    if model_type != "bert":
                                        result_data['prompt_name'] = os.path.basename(subdir)
                                    else:
                                        result_data['prompt_name'] = None
                                    
                                    results.append(result_data)

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Specify the order of the columns
    columns_first = ['dataset_name', 'model_type', 'model_name', 'classifier']
    if 'prompt_name' in results_df.columns:
        columns_first.append('prompt_name')
    column_order = columns_first + [col for col in results_df.columns if col not in columns_first]
    results_df = results_df[column_order]
    
    return results_df

In [22]:
base_path = 'results' 

df = load_results_to_dataframe(base_path)
df

,dataset_name,model_type,model_name,classifier,prompt_name,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,split4_test_f1_score,mean_test_f1_score,embedding_generation_time,embedding_generation_size
0,Dmoz-Science.csv,ollama,mistral:7b,knn,instruction_summary_prompt,0.593222,0.592466,0.602268,0.561719,0.578065,0.585548,NaN,None
1,Dmoz-Science.csv,ollama,mistral:7b,knn,instruction_classification_prompt,0.629638,0.649192,0.665732,0.634082,0.634035,0.642536,NaN,None
2,Dmoz-Science.csv,ollama,mistral:7b,knn,base_prompt,0.537706,0.551799,0.563238,0.549503,0.546192,0.549688,NaN,None
3,Dmoz-Science.csv,ollama,qwen2.5:7b,knn,instruction_summary_prompt,0.652639,0.667422,0.697797,0.665265,0.671679,0.670960,NaN,None
4,Dmoz-Science.csv,ollama,qwen2.5:7b,knn,instruction_classification_prompt,0.688759,0.694056,0.700061,0.680398,0.679367,0.688528,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,SyskillWebert.csv,llm2vec,McGill-NLP_LLM2Vec-Mistral-7B-Instruct-v2-mntp...,knn,instruction_classification_prompt,0.807679,0.913998,0.822170,0.822727,0.862566,0.845828,NaN,None
370,SyskillWebert.csv,llm2vec,McGill-NLP_LLM2Vec-Mistral-7B-Instruct-v2-mntp...,knn,base_prompt,0.892391,0.917844,0.822991,0.887147,0.825480,0.869171,NaN,None
371,SyskillWebert.csv,llm2vec,McGill-NLP_LLM2Vec-Sheared-LLaMA-mntp-supervised,knn,instruction_summary_prompt,0.823295,0.897993,0.952120,0.907743,0.921861,0.900603,NaN,None
372,SyskillWebert.csv,llm2vec,McGill-NLP_LLM2Vec-Sheared-LLaMA-mntp-supervised,knn,instruction_classification_prompt,0.733333,0.807781,0.794508,0.815568,0.798438,0.789926,NaN,None


# Desempenho

In [23]:
import pandas as pd

# Defina os splits que você quer combinar
splits = ['split0_test_f1_score', 
          'split1_test_f1_score', 
          'split2_test_f1_score', 
          'split3_test_f1_score', 
          'split4_test_f1_score']

# Agrupar pelo 'prompt_name' e combinar os splits em uma lista para cada 'prompt_name'
df_combined = df.groupby('model_type')[splits].apply(lambda x: x.values.flatten().tolist()).reset_index()

# Exibir o resultado
df_combined  # A coluna 0 contém a lista dos splits combinados


,model_type,0
0,bert,"[0.7814881599520914, 0.7814233885535803, 0.807..."
1,llm2vec,"[0.8063511747045715, 0.7997685567439344, 0.840..."
2,ollama,"[0.5932220682569391, 0.5924657799692853, 0.602..."


In [24]:
bert = df_combined[df_combined['model_type'] == "bert"][0].tolist()[0]
llm2vec = df_combined[df_combined['model_type'] == "llm2vec"][0].tolist()[0]

In [25]:
from scipy import stats

#Se o p-valor for maior que 0,05: Os dados não apresentam evidências suficientes para rejeitar a hipótese de normalidade, ou seja, os dados podem ser considerados normalmente distribuídos.
#Se o p-valor for menor que 0,05: Os dados provavelmente não seguem uma distribuição normal, o que pode indicar a necessidade de ajustes ou o uso de testes não paramétricos.

# Teste de Shapiro-Wilk para normalidade
shapiro_bert_stat, shapiro_bert_p = stats.shapiro(bert)
shapiro_llm2vec_stat, shapiro_llm2vec_p = stats.shapiro(llm2vec)

# Imprimir os resultados do teste de Shapiro-Wilk
print(f'Teste de Shapiro-Wilk para BERT: Estatística = {shapiro_bert_stat}, p-valor = {shapiro_bert_p}')
print(f'Teste de Shapiro-Wilk para LLM2Vec: Estatística = {shapiro_llm2vec_stat}, p-valor = {shapiro_llm2vec_p}')

Teste de Shapiro-Wilk para BERT: Estatística = 0.9261211660374754, p-valor = 4.718960862983999e-09
Teste de Shapiro-Wilk para LLM2Vec: Estatística = 0.9394682999067311, p-valor = 1.199433054295266e-19


In [26]:
from scipy.stats import kruskal

# Executando o teste de Kruskal-Wallis
stat, p_value = kruskal(bert, llm2vec)

# Exibindo o resultado
print(f"Estatística de Kruskal-Wallis: {stat}")
print(f"Valor-p: {p_value}")

Estatística de Kruskal-Wallis: 1.8606317712580547
Valor-p: 0.1725519025126044


In [28]:
from scipy.stats import rankdata
import numpy as np

# Cálculo das medianas para BERT e LLM2Vec
mediana_bert = np.median(bert)
mediana_llm2vec = np.median(llm2vec)

print(f'Mediana BERT: {mediana_bert}')
print(f'Mediana LLM2Vec: {mediana_llm2vec}')


Mediana BERT: 0.8711143109091124
Mediana LLM2Vec: 0.8776299372374904


# Prompt

In [29]:
import pandas as pd

# Defina os splits que você quer combinar
splits = ['split0_test_f1_score', 
          'split1_test_f1_score', 
          'split2_test_f1_score', 
          'split3_test_f1_score', 
          'split4_test_f1_score']

# Agrupar pelo 'prompt_name' e combinar os splits em uma lista para cada 'prompt_name'
df_combined = df.groupby('prompt_name')[splits].apply(lambda x: x.values.flatten().tolist()).reset_index()

# Exibir o resultado
df_combined  # A coluna 0 contém a lista dos splits combinados


,prompt_name,0
0,base_prompt,"[0.5377064641913295, 0.5517986440058487, 0.563..."
1,instruction_classification_prompt,"[0.6296381839440847, 0.6491923816459985, 0.665..."
2,instruction_summary_prompt,"[0.5932220682569391, 0.5924657799692853, 0.602..."


In [30]:
base_prompt = df_combined[df_combined['prompt_name'] == "base_prompt"][0].tolist()[0]
instruction_summary_prompt = df_combined[df_combined['prompt_name'] == "instruction_summary_prompt"][0].tolist()[0]
instruction_classification_prompt = df_combined[df_combined['prompt_name'] == "instruction_classification_prompt"][0].tolist()[0]

In [31]:
from scipy.stats import shapiro

#Se o p-valor for maior que 0,05: Os dados não apresentam evidências suficientes para rejeitar a hipótese de normalidade, ou seja, os dados podem ser considerados normalmente distribuídos.
#Se o p-valor for menor que 0,05: Os dados provavelmente não seguem uma distribuição normal, o que pode indicar a necessidade de ajustes ou o uso de testes não paramétricos.

# Teste de Shapiro-Wilk
shapiro_base = shapiro(base_prompt)
shapiro_summary = shapiro(instruction_summary_prompt)
shapiro_classification = shapiro(instruction_classification_prompt)

print("Shapiro-Wilk:")
print("base_prompt:", shapiro_base)
print("instruction_summary_prompt:", shapiro_summary)
print("instruction_classification_prompt:", shapiro_classification)

Shapiro-Wilk:
base_prompt: ShapiroResult(statistic=np.float64(0.9651226999400369), pvalue=np.float64(3.827298012360926e-10))
instruction_summary_prompt: ShapiroResult(statistic=np.float64(0.972072471875919), pvalue=np.float64(9.80213675867632e-09))
instruction_classification_prompt: ShapiroResult(statistic=np.float64(0.9708462662678546), pvalue=np.float64(5.351582475352289e-09))


In [32]:
from scipy.stats import kruskal

# Executando o teste de Kruskal-Wallis
stat, p_value = kruskal(base_prompt, instruction_summary_prompt, instruction_classification_prompt)

# Exibindo o resultado
print(f"Estatística de Kruskal-Wallis: {stat}")
print(f"Valor-p: {p_value}")

Estatística de Kruskal-Wallis: 0.08170979181693862
Valor-p: 0.9599684151869093


# Tempo

In [33]:
bert = df[df['model_type'] == 'bert']['embedding_generation_time']
llm2vec = df[df['model_type'] == 'llm2vec']['embedding_generation_time']

In [34]:
from scipy import stats

#Se o p-valor for maior que 0,05: Os dados não apresentam evidências suficientes para rejeitar a hipótese de normalidade, ou seja, os dados podem ser considerados normalmente distribuídos.
#Se o p-valor for menor que 0,05: Os dados provavelmente não seguem uma distribuição normal, o que pode indicar a necessidade de ajustes ou o uso de testes não paramétricos.

# Teste de Shapiro-Wilk para normalidade
shapiro_bert_stat, shapiro_bert_p = stats.shapiro(bert)
shapiro_llm2vec_stat, shapiro_llm2vec_p = stats.shapiro(llm2vec)

# Imprimir os resultados do teste de Shapiro-Wilk
print(f'Teste de Shapiro-Wilk para BERT: Estatística = {shapiro_bert_stat}, p-valor = {shapiro_bert_p}')
print(f'Teste de Shapiro-Wilk para LLM2Vec: Estatística = {shapiro_llm2vec_stat}, p-valor = {shapiro_llm2vec_p}')

Teste de Shapiro-Wilk para BERT: Estatística = nan, p-valor = nan
Teste de Shapiro-Wilk para LLM2Vec: Estatística = nan, p-valor = nan


In [35]:
from scipy.stats import kruskal

# Executando o teste de Kruskal-Wallis
stat, p_value = kruskal(bert, llm2vec)

# Exibindo o resultado
print(f"Estatística de Kruskal-Wallis: {stat}")
print(f"Valor-p: {p_value}")

Estatística de Kruskal-Wallis: nan
Valor-p: nan


In [36]:
from scipy.stats import rankdata
import numpy as np

# Cálculo das medianas para BERT e LLM2Vec
mediana_bert = np.median(bert)
mediana_llm2vec = np.median(llm2vec)

print(f'Mediana BERT: {mediana_bert}')
print(f'Mediana LLM2Vec: {mediana_llm2vec}')

# Comparando as medianas
if mediana_bert < mediana_llm2vec:
    print("O modelo BERT tem os menores valores de tempo de geração de embeddings.")
else:
    print("O modelo LLM2Vec tem os menores valores de tempo de geração de embeddings.")


Mediana BERT: nan
Mediana LLM2Vec: nan
O modelo LLM2Vec tem os menores valores de tempo de geração de embeddings.
